In [1]:
%load_ext autoreload
%autoreload 2

In [44]:
import torch
import matplotlib.pyplot as plt
from botorch.utils.sampling import draw_sobol_samples 
import numpy as np 
import json

import sys
sys.path.append('..')
import sample_utilities.samples as samples
import uuid

import scipy
from scipy.stats.qmc import Sobol
from scipy.stats import qmc

import pandas as pd

## Load bounds

In [16]:
target_volume = 10000

In [17]:
with open('Mesoporous_constants_APS.json', 'r') as f:
    constants = json.load(f)


ctab_lower_vf = constants['ctab']['minimum_mass']/constants['ctab']['stock_concentration_mg_uL']/target_volume  
f127_lower_vf = constants['f127']['minimum_mass']/constants['f127']['stock_concentration_mg_uL']/target_volume

ctab_upper_vf = constants['ctab']['maximum_mass']/constants['ctab']['stock_concentration_mg_uL']/target_volume
f127_upper_vf = constants['f127']['maximum_mass']/constants['f127']['stock_concentration_mg_uL']/target_volume

lower_bounds = [constants['TEOS']['minimum_volume_fraction'],
                constants['ammonia']['minimum_volume_fraction'],
                constants['ethanol']['minimum_volume_fraction'],
                constants['ctab']['minimum_mass'],
                constants['f127']['minimum_mass']]

upper_bounds = [constants['TEOS']['maximum_volume_fraction'],
                constants['ammonia']['maximum_volume_fraction'],
                constants['ethanol']['maximum_volume_fraction'],
                constants['ctab']['maximum_mass'],
                constants['f127']['maximum_mass']]

In [18]:
ctab_upper_vf

0.6666666666666667

In [25]:
def constrained_sobol_sampling_rejection(n: int, num_samples: int, m_samples = 8, seed: int = None):
    """
    Generates Sobol samples in n dimensions satisfying:
    a) Sum of all components = 1.0
    b) Last component (nth dim) is in [0.4, 1.0]

    Uses rejection sampling: Generates a large pool and filters valid samples.

    Args:
        n (int): Number of dimensions.
        num_samples (int): Number of valid samples to return.
        pool_size (int): Number of initial samples to generate before filtering.
        seed (int, optional): Random seed for reproducibility.

    Returns:
        torch.Tensor: (num_samples, n) tensor of valid samples.
    """
    if seed is not None:
        torch.manual_seed(seed)

    valid_samples = []
    
    while len(valid_samples) < num_samples:
        print('Number valid samples: ', len(valid_samples))
        # Generate Sobol samples in [0,1]^n
        #sobol_samples = draw_sobol_samples(bounds=torch.tensor([[0.0] * n, [1.0] * n]), n=pool_size, q=1).squeeze(1)
        sampler = Sobol(d=n, seed = seed)
        sampled_points = sampler.random_base2(m_samples)
    
        sampled_volume_fractions = qmc.scale(sampled_points, lower_bounds, upper_bounds)


        for i in range(len(sampled_volume_fractions)):
            teos_vol_frac = sampled_volume_fractions[i, 0]
            ammonia_vol_frac = sampled_volume_fractions[i, 1]
            ethanol_vol_frac = sampled_volume_fractions[i, 2]
            # convert surfactant mass to volumes
            ctab_mass = sampled_volume_fractions[i, 3]
            f127_mass = sampled_volume_fractions[i, 4]
    
            ctab_vol_frac = ctab_mass/constants['ctab']['stock_concentration_mg_uL']/target_volume
            f127_vol_frac = f127_mass/constants['f127']['stock_concentration_mg_uL']/target_volume
    
            water_from_surfactant_vf = ctab_vol_frac+f127_vol_frac
    
            
    
            water_min_vf = constants['water']['minimum_volume_fraction']
    
            # water that needs to be delivered to meet minimum volume fraction
            water_min_vf_delivered = max(water_min_vf-water_from_surfactant_vf, 0)
    
            max_other_components = 1 - water_min_vf_delivered

            print(max_other_components)
    
    
            sum_nonwater_additions = teos_vol_frac + ammonia_vol_frac + ethanol_vol_frac + ctab_vol_frac + f127_vol_frac
            sum_water_free_additions = teos_vol_frac + ammonia_vol_frac + ethanol_vol_frac
            if sum_nonwater_additions < max_other_components:

                water_vol_frac = 1 - sum_water_free_additions
                sample = [teos_vol_frac, ammonia_vol_frac, ethanol_vol_frac, ctab_mass, f127_mass, water_vol_frac]
                
                print('found valid sample: ', sample)
                valid_samples.append(sample)

    return valid_samples[0:num_samples]


In [26]:
baseline_samples = constrained_sobol_sampling_rejection(5, 32, m_samples= 7)

Number valid samples:  0
1
found valid sample:  [np.float64(0.05304859378188849), np.float64(0.021066191149875523), np.float64(0.34019104645587506), np.float64(71.31070161238313), np.float64(33.352832682430744), np.float64(0.5856941686123609)]
1
1
1
found valid sample:  [np.float64(0.07990537553001195), np.float64(0.07631106785498559), np.float64(0.20112086267210544), np.float64(48.75404614955187), np.float64(107.18125300481915), np.float64(0.642662693942897)]
1
found valid sample:  [np.float64(0.06897758682211862), np.float64(0.052251707557588824), np.float64(0.2625954675395042), np.float64(35.794096160680056), np.float64(116.36743415147066), np.float64(0.6161752380807883)]
1
1
found valid sample:  [np.float64(0.03452036137832329), np.float64(0.021927289711311464), np.float64(0.23905928236432375), np.float64(30.59445008635521), np.float64(234.31539237499237), np.float64(0.7044930665460415)]
1
found valid sample:  [np.float64(0.04219471870781854), np.float64(0.08727075284346938), np.fl

In [27]:
for sample in baseline_samples:
    sample_sum = 0
    for entry in sample:
        sample_sum += entry
    print('Sample sum: ', sample_sum)


Sample sum:  105.66353429481387
Sample sum:  156.93529915437102
Sample sum:  153.16153031215072
Sample sum:  265.9098424613476
Sample sum:  101.70367557927966
Sample sum:  44.36378931067884
Sample sum:  147.5983222331852
Sample sum:  199.70597490109503
Sample sum:  123.82351589761674
Sample sum:  125.92781786248088
Sample sum:  139.42312609776855
Sample sum:  140.97375493869185
Sample sum:  24.06602976284921
Sample sum:  84.73353770934045
Sample sum:  56.130467815324664
Sample sum:  202.59359687007964
Sample sum:  56.93519913963973
Sample sum:  167.25658774748445
Sample sum:  90.7750892676413
Sample sum:  86.66770704463124
Sample sum:  189.50557464174926
Sample sum:  88.39794026128946
Sample sum:  70.84256039373577
Sample sum:  233.42445883341134
Sample sum:  186.21561631932855
Sample sum:  263.3569232970476
Sample sum:  83.84827487543225
Sample sum:  128.04843120649457
Sample sum:  133.2100734245032
Sample sum:  135.32519818656147
Sample sum:  105.72431415691972
Sample sum:  27.923215

In [28]:
len(baseline_samples)

32

In [11]:
uuid_vals = [uuid.uuid4() for val in baseline_samples]
teos_volume = [target_volume*entry[0] for entry in baseline_samples]

In [40]:
samples_gen = []
for i in range(len(baseline_samples)):
    row = baseline_samples[i]
    sample = samples.MesoporousSample(target_volume=target_volume, reactant_fp='Mesoporous_constants_APS.json', teos_vol_frac=row[0], ammonia_vol_frac=row[1], ethanol_vol_frac=row[2], ctab_mass=row[3], f127_mass=row[4], water_vol_frac=row[5])
    sample.calculate_reactant_volumes()
    #sample.calculate_silica_mass_concentration()
    #sample.calculate_silica_mass_fraction()
    #sample.calculate_dilution_volumefraction(target_dilution)
    samples_gen.append(sample)
    print('final etoh vol: ', sample.ethanol_volume)

TEOS ETOH:  981.3989849649371
Solvent etoh total:  981.3989849649371
final etoh vol:  2420.5114795938134
TEOS ETOH:  1478.249447305221
Solvent etoh total:  1478.249447305221
final etoh vol:  532.9591794158332
TEOS ETOH:  1276.0853562091947
Solvent etoh total:  1276.0853562091947
final etoh vol:  1349.869319185847
TEOS ETOH:  638.6266854989808
Solvent etoh total:  638.6266854989808
final etoh vol:  1751.9661381442565
TEOS ETOH:  780.602296094643
Solvent etoh total:  780.602296094643
final etoh vol:  2980.700471319957
TEOS ETOH:  742.5868363361806
Solvent etoh total:  742.5868363361806
final etoh vol:  3173.8180681820963
TEOS ETOH:  247.1238580965437
Solvent etoh total:  247.1238580965437
final etoh vol:  2303.700903387275
TEOS ETOH:  1319.6130257027692
Solvent etoh total:  1319.6130257027692
final etoh vol:  2184.5955417563673
TEOS ETOH:  585.4301863585133
Solvent etoh total:  585.4301863585133
final etoh vol:  2629.086222103564
TEOS ETOH:  1081.6456668260507
Solvent etoh total:  1081.6

In [41]:
sample.water_volume

np.float64(3338.890667854186)

In [49]:
uuid_vals = []
teos_volumes = []
ammonia_volumes = []
ethanol_volumes = []
water_volumes = []
ctab_volumes = []
f127_volumes = []
sample_names = []

for i, sample in enumerate(samples_gen):
    sample_names.append(f'SobolBaseline_{i+1}')
    uuid_vals.append(uuid.uuid4())
    teos_volumes.append(sample.teos_volume)
    ammonia_volumes.append(sample.ammonia_volume)
    ethanol_volumes.append(sample.ethanol_volume)
    water_volumes.append(sample.water_volume)
    ctab_volumes.append(sample.ctab_volume)
    f127_volumes.append(sample.f127_volume)\

    total_volume = sample.teos_volume+sample.ammonia_volume + sample.water_volume + sample.ethanol_volume + sample.ctab_volume + sample.f127_volume
    print(f'Total volume sample SobolBaseline_{i+1}: {total_volume}')



Total volume sample SobolBaseline_1: 10000.0
Total volume sample SobolBaseline_2: 10000.0
Total volume sample SobolBaseline_3: 10000.0
Total volume sample SobolBaseline_4: 10000.0
Total volume sample SobolBaseline_5: 10000.0
Total volume sample SobolBaseline_6: 10000.0
Total volume sample SobolBaseline_7: 10000.0
Total volume sample SobolBaseline_8: 10000.0
Total volume sample SobolBaseline_9: 10000.0
Total volume sample SobolBaseline_10: 10000.0
Total volume sample SobolBaseline_11: 10000.0
Total volume sample SobolBaseline_12: 10000.0
Total volume sample SobolBaseline_13: 9999.999999999998
Total volume sample SobolBaseline_14: 9999.999999999998
Total volume sample SobolBaseline_15: 10000.0
Total volume sample SobolBaseline_16: 10000.0
Total volume sample SobolBaseline_17: 10000.0
Total volume sample SobolBaseline_18: 10000.0
Total volume sample SobolBaseline_19: 10000.0
Total volume sample SobolBaseline_20: 10000.0
Total volume sample SobolBaseline_21: 10000.0
Total volume sample Sob

In [54]:
sample_table = pd.DataFrame({'sample_name':sample_names,
                             'uuid':uuid_vals,
                             'teos_volume':teos_volumes,
                             'ammonia_volume':ammonia_volumes,
                             'water_volume':water_volumes,
                             'ethanol_volume':ethanol_volumes,
                             'ctab_volume':ctab_volumes,
                             'F127_volume':f127_volumes
                            })

In [55]:
sample_table

,sample_name,uuid,teos_volume,ammonia_volume,water_volume,ethanol_volume,ctab_volume,F127_volume
0,SobolBaseline_1,a80865b1-36ed-4baf-a7d8-d28a199bb63b,1511.884923,210.661911,435.838258,2420.511480,4754.046774,667.056654
1,SobolBaseline_2,ba434c27-0b46-4f06-9c76-28ee87a75ac1,2277.303203,763.110679,1032.732136,532.959179,3250.269743,2143.625060
2,SobolBaseline_3,779eda3f-d4ba-4cbc-8245-67ac09673394,1965.861224,522.517076,1448.130620,1349.869319,2386.273077,2327.348683
3,SobolBaseline_4,e3fd154b-26b4-41a7-beaf-eeee4ba3a623,983.830299,219.272897,318.992812,1751.966138,2039.630006,4686.307847
4,SobolBaseline_5,133a1284-ad26-4069-bf18-f06d4ab5e1b4,1202.549483,872.707528,215.617315,2980.700471,3877.645272,850.779930
5,SobolBaseline_6,9ee884ca-55d5-4fbf-a0a9-beffcd6e44b3,1143.985126,697.689312,2192.589307,3173.818068,2749.489144,42.429043
6,SobolBaseline_7,2e8e5baa-644b-4f05-8254-6df9d9dd3c95,380.704322,134.805263,1279.996186,2303.700903,4241.181259,1659.612067
7,SobolBaseline_8,f48f314e-988e-4387-92bd-916d91454f96,2032.917364,294.917703,842.584337,2184.595542,958.379366,3686.605688
8,SobolBaseline_9,4d37484d-1de7-4060-b485-fc09c3a8a452,901.878936,489.171336,961.301699,2629.086222,3660.130699,1358.431108
9,SobolBaseline_10,6a757874-6ae6-4565-9e76-56e2047c6122,1666.319000,455.221368,1148.582722,2865.645471,1950.964401,1913.267037


In [56]:
vial_volume = 17000
cols = ['teos_volume', 'ammonia_volume', 'ethanol_volume', 'water_volume', 'ctab_volume', 'F127_volume']
for col in cols:
    total_volume = sample_table[col].sum()
    n_vials = total_volume/vial_volume
    print(f'Volume for {col}: {total_volume}, vials required: {n_vials}')

Volume for teos_volume: 37330.548830148065, vials required: 2.195914637067533
Volume for ammonia_volume: 15665.848736092448, vials required: 0.9215205138877911
Volume for ethanol_volume: 67845.74454808678, vials required: 3.9909261498874575
Volume for water_volume: 60718.44629828041, vials required: 3.5716733116635537
Volume for ctab_volume: 80965.70286465189, vials required: 4.762688403803052
Volume for F127_volume: 57473.70872274041, vials required: 3.3808063954553185


In [57]:
sample_table.to_csv('Mesoporous_SobolBaseline_APS_3_28_25.csv')